In [17]:
from selenium import webdriver
import pandas as pd
import numpy as np
import bs4 as bs
import requests
import time
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import sqlite3

import json


# Scrapes club information from https://win.wisc.edu/organizations

In [3]:
orgs = {} # category/links to orgs

In [4]:

script = """
window.initialAppState = {
  "institution": null,
  "coverPhoto": {
    "id": 33292,
    "imageId": "3efac84c-3a41-4576-b8a2-954ac060ab54",
    "imagePath": "1a2713b8-ac31-4b8d-80ea-5bedc778fe98601cb149-9070-4029-bdce-358e6f49baf2.png",
    "url": "https://se-images-blob.campuslabs.com/communities/13/coverphotos/3efac84c-3a41-4576-b8a2-954ac060ab54.jpg",
    "thumbnailUrl": "https://se-images-blob.campuslabs.com/communities/13/coverphotos/3efac84c-3a41-4576-b8a2-954ac060ab54-thumbnail.jpg",
    "caption": "Uploaded cover photo has no appropriate alternative text",
    "dateCreated": "2015-09-18T11:00:19-04:00",
    "institutionId": 13
  },
  "user": null,
  "preFetchedData": {
    "event": null,
    "loadAdditionalEventDetails": false,
    "organization": null,
    "article": null
  },
  "imageServerBaseUrl": "https://se-images.campuslabs.com/clink/images/",
  "baseUrl": "https://win.wisc.edu",
  "serverSideContextRoot": "",
  "serviceOpportunitiesEnabled": true,
  "xsrfToken": "CfDJ8Az9GcAD4TpIm2NmyW6ywWGVx-h6W6ReMiMmwNyNFfXNvjAWSEeM9-k4BTBtxd3GDZfDT9totomVmjvNV_EL8PRB2ENuQ8UMNjI02Yp4VUMkTjSkJO59zbEoRZyxcskyor4fICpUSLyt57P7fBKo_Qs",
  "reCaptchaPublicKey": "6LdlpB0TAAAAAJ25ETYC1pRiwGiT7clzxta-HGH0",
  "returnUrl": "",
  "launchDarkly": null
};
"""
    
category_ids = {10879: 'Academic/Career', 10877: 'Activism/Advocacy', 10875: 'Agricultural/Environmental', 2462: 'Arts/Music', 2463: 'Cultural/Ethnic', 10874: 'Graduate/Professional', 10873: 'Health/Wellness', 2464: 'Honorary', 10876: 'Media/Publication', 10878: 'Political Interest', 2467: 'Religious/Spiritual', 2469: 'Service/Volunteer', 2470: 'Social Fraternity/Sorority', 2471: 'Sports/Recreation'}
    
for category_id in category_ids:
  url = f"https://win.wisc.edu/organizations?categories={category_id}"
  driver = webdriver.Chrome()
  driver.get(url)
  time.sleep(2)
  buttons = driver.find_elements(By.CLASS_NAME, 'outlinedButton')
  while len(buttons) > 1:
      buttons[1].click()
      time.sleep(1)
      buttons = driver.find_elements(By.CLASS_NAME, 'outlinedButton')

  div_element = driver.find_element(By.ID, 'org-search-results')
  soup = bs.BeautifulSoup(div_element.get_attribute('innerHTML'), 'html.parser')
  link_elements = soup.find_all('a')

  links = []
  for link_element in link_elements:
      if link_element:
        link_url = link_element.get('href')  # Get the link's URL
        if link_url:
            links.append(link_url)
      
    
  orgs[category_ids[category_id]] = links
 

In [6]:
  # Put all links into club_categories
  conn = sqlite3.connect('clubs.db')
  c = conn.cursor()
  for category_id in category_ids:
    for link in orgs[category_ids[category_id]]:
      c.execute("INSERT INTO club_categories VALUES (?, ?)", (link, category_id))
      
  
  conn.commit()
  conn.close()

In [49]:
clubs = pd.read_sql("SELECT * FROM club_categories", conn)
club_ids = clubs['club_id'].unique()
for club_id in club_ids:
    link = club_id
    url = f'https://win.wisc.edu{link}'
    
    

In [26]:
url = 'https://win.wisc.edu/organization/acssc-uwm'
driver = webdriver.Chrome()
driver.get(url)
soup = bs.BeautifulSoup(driver.page_source, 'html.parser')

# Find script with window.initialAppState
scripts = soup.find_all('script')
for script in scripts:
    if script.text.startswith('window.initialAppState'):
        script_text = script.text[:-1]
        script_text = script_text.replace('window.initialAppState = ', '')
        break
 

# Get socials from json


In [27]:

data = json.loads(script_text.replace('window.initialAppState = ', ''))

data['preFetchedData']['organization']['socialMedia']



{'externalWebsite': 'https://acsscuwm.weebly.com/',
 'flickrUrl': '',
 'googleCalendarUrl': '',
 'googlePlusUrl': '',
 'instagramUrl': 'https://www.instagram.com/acssc_uw/',
 'linkedInUrl': '',
 'pinterestUrl': '',
 'tumblrUrl': '',
 'vimeoUrl': '',
 'youtubeUrl': '',
 'facebookUrl': 'https://www.facebook.com/American-Cetacean-Society-Student-Coalition-UW-Madison-1520973208003540/',
 'twitterUrl': None,
 'twitterUserName': 'acssc_uw'}